In [1]:
import os, sys
sys.path.append("../../common/")
from python_tools import *

python tools loaded.


In [2]:
%matplotlib notebook

In [3]:
# some useful functions
# 
# you may need to comment out the 'numba' bits if your system can't install numba (like the gpvms...)
#

@numba.jit
def costheta_numba(p1x,p1y,p1z,p1mag,
                   p2x,p2y,p2z,p2mag):
    return np.clip(np.where((p1mag>0.0)&(p2mag>0.0),
                            (p1x*p2x+p1y*p2y+p1z*p2z)/p1mag/p2mag,
                            np.nan),
                   -1.0,1.0)

def eval_costheta(df,suffix1="",suffix2=""):
    return costheta_numba(df.loc[:,"px"+suffix1].values,df.loc[:,"py"+suffix1].values,df.loc[:,"pz"+suffix1].values,df.loc[:,"p"+suffix1].values,
                          df.loc[:,"px"+suffix2].values,df.loc[:,"py"+suffix2].values,df.loc[:,"pz"+suffix2].values,df.loc[:,"p"+suffix2].values)

    
@numba.jit(nopython=True)
def q3_numba(p1x,p1y,p1z,p2x,p2y,p2z):
    return np.sqrt((p1x-p2x)**2+(p1y-p2y)**2+(p1z-p2z)**2)

def eval_q3(df,suffix1="",suffix2="_mu"):
    return q3_numba(df.loc[:,"px"+suffix1].values,df.loc[:,"py"+suffix1].values,df.loc[:,"pz"+suffix1].values,
                    df.loc[:,"px"+suffix2].values,df.loc[:,"py"+suffix2].values,df.loc[:,"pz"+suffix2].values)

In [4]:
root_filenames = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/Run3/*/*.root")

In [5]:
#Read from files (ROOT or HDF5)

t_df = []
p_df = []
pot_df = []

file_count = 0
event_count = 0
print("Processing %d files" % len(root_filenames))

for root_filename in root_filenames:
    
    try:
        p_df.append(uproot.open(root_filename)['mcana/particle_tree'].pandas.df())
        t_df.append(uproot.open(root_filename)['mcana/mctruth_tree'].pandas.df())
        pot_df.append(uproot.open(root_filename)['potana/pot_tree'].pandas.df())
    except:
        print("File %s, trees not found."%root_filename)
    
    event_count += len(t_df[-1])
    file_count += 1
    if file_count%500==0:
        print("\tProcessed %d files. %d events processed." % (file_count,event_count))

p_df = pd.concat(p_df)
t_df = pd.concat(t_df)
pot_df = pd.concat(pot_df)

p_df.set_index(["run","subrun","event","truth_index","p_index"],inplace=True)
t_df.set_index(["run","subrun","event","truth_index"],inplace=True)
pot_df.set_index(["run","subrun"],inplace=True)
        
print("Have dataframe objects. Total events is %d." % len(t_df))

Processing 23086 files
	Processed 500 files. 9045 events processed.
	Processed 1000 files. 18303 events processed.
	Processed 1500 files. 27356 events processed.
	Processed 2000 files. 36163 events processed.
File /Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/Run3/32044093_1706/sampler_hist_1706.root, trees not found.
	Processed 2500 files. 45546 events processed.
	Processed 3000 files. 54681 events processed.
	Processed 3500 files. 63953 events processed.
	Processed 4000 files. 73347 events processed.
File /Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/Run3/32044121_1183/sampler_hist_1183.root, trees not found.
	Processed 4500 files. 82819 events processed.
	Processed 5000 files. 92318 events processed.
	Processed 5500 files. 101552 events processed.
	Processed 6000 files. 110767 events processed.
	Processed 6500 files. 120243 events processed.
	Processed 7000 files. 129437 events processed.
	Processed 7500 files. 138944 events processed.
	Processed 8000 files. 148457 events pr

In [6]:
# calculate integrated POT for all the events we have

TOTAL_POT = pot_df["totpot"].sum()
TOTAL_EVENTS = len(t_df)
print("\n\n")
print("Total events of %d in POT of %E. Events per 1e20 POT is %f" % (TOTAL_EVENTS,
                                                                      TOTAL_POT,
                                                                      TOTAL_EVENTS/(TOTAL_POT/1e20)))
print("\n\n")




Total events of 425382 in POT of 4.113101E+20. Events per 1e20 POT is 103421.244025





In [9]:
p_df_numu = p_df.query("status==0 and pdgcode==14").groupby(["run","subrun","event","truth_index"]).first()
p_df_nue = p_df.query("status==0 and pdgcode==12").groupby(["run","subrun","event","truth_index"]).first()

In [11]:
print(len(p_df_numu))
print(len(p_df_nue))      

419100
2544


In [12]:
p_df_nue

status  trackid  pdgcode  mother  \
run      subrun event truth_index                                     
32044041 11     5     0                 0        0       12      -1   
         20     2     0                 0        0       12      -1   
         23     4     0                 0        0       12      -1   
         37     2     0                 0        0       12      -1   
         46     7     0                 0        0       12      -1   
...                                   ...      ...      ...     ...   
32044121 3099   4     0                 0        0       12      -1   
         3102   14    0                 0        0       12      -1   
                20    0                 0        0       12      -1   
         3104   4     0                 0        0       12      -1   
         3105   26    0                 0        0       12      -1   

                                      process endprocess  weight  rescatter  \
run      subrun event truth_index                                             
32044041 11     5     0            b'primary'        b''     0.0         -1   
         20     2     0            b'primary'        b''     0.0         -1   
         23     4     0            b'primary'        b''     0.0         -1   
         37     2     0            b'primary'        b''     0.0         -1   
         46     7     0            b'primary'        b''     0.0         -1   
...                                       ...        ...     ...        ...   
32044121 3099   4     0            b'primary'        b''     0.0         -1   
         3102   14    0            b'primary'        b''     0.0         -1   
                20    0            b'primary'        b''     0.0         -1   
         3104   4     0            b'primary'        b''     0.0         -1   
         3105   26    0            b'primary'        b''     0.0         -1   

                                      start_x     start_y  ...        py  \
run      subrun event truth_index                          ...             
32044041 11     5     0            146.293991 -112.855087  ... -0.002532   
         20     2     0             66.051933   40.338955  ... -0.000979   
         23     4     0             65.104813 -134.535751  ... -0.003868   
         37     2     0            168.178925   90.327751  ...  0.003152   
         46     7     0            104.169495 -109.827881  ... -0.003449   
...                                       ...         ...  ...       ...   
32044121 3099   4     0             -4.844387   51.573830  ... -0.000601   
         3102   14    0            205.389969  -62.930603  ... -0.004279   
                20    0            176.041412  -94.596397  ... -0.002928   
         3104   4     0            108.889954   49.742649  ... -0.000813   
         3105   26    0             10.582222 -146.955826  ... -0.001082   

                                         pz         p        pt         e  \
run      subrun event truth_index                                           
32044041 11     5     0            0.767868  0.767878  0.003901  0.767878   
         20     2     0            1.046259  1.046260  0.001292  1.046260   
         23     4     0            0.596402  0.596417  0.004106  0.596417   
         37     2     0            1.575546  1.575556  0.005599  1.575556   
         46     7     0            0.628561  0.628574  0.003990  0.628574   
...                                     ...       ...       ...       ...   
32044121 3099   4     0            1.785686  1.785687  0.001914  1.785687   
         3102   14    0            1.506153  1.506179  0.008825  1.506179   
                20    0            1.130455  1.130470  0.005925  1.130470   
         3104   4     0            3.035096  3.035105  0.007223  3.035105   
         3105   26    0            0.220019  0.220022  0.001120  0.220022   

                                   mass    end_px    end_py    end_pz  \
run      subrun eve

In [ ]:
df_n = pd.DataFrame()
df_n["is_mu"] = ((p_df["status"]==1)&(p_df["pdgcode"]==13)).replace(False,np.nan)
df_n["is_p_30MeV"] = ((p_df["status"]==1)&(p_df["pdgcode"]==2212)&((p_df["e"]-p_df["mass"])>0.03)).replace(False,np.nan)
df_n["is_pi"] = ((p_df["status"]==1)&((p_df["pdgcode"]==211)^(p_df["pdgcode"]==-211)^(p_df["pdgcode"]==111))).replace(False,np.nan)
df_n = df_n.groupby(["run","subrun","event","truth_index"]).agg("sum").rename(columns={"is_mu":"n_mu","is_p_30MeV":"n_p_30MeV","is_pi":"n_pi"})

In [ ]:
p_df_mu = p_df.query("status==1 and pdgcode==13").groupby(["run","subrun","event","truth_index"]).first()
p_df_nu = p_df.query("status==0 and pdgcode==14").groupby(["run","subrun","event","truth_index"]).first()

In [ ]:
df_ev_t = t_df.copy()
df_ev_t = df_ev_t.merge(p_df_nu,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_nu"])
df_ev_t = df_ev_t.merge(p_df_mu,how="left",on=["run","subrun","event","truth_index"],suffixes=["","_mu"])
df_ev_t = df_ev_t.merge(df_n,how="left",on=["run","subrun","event","truth_index"])

In [ ]:
df_ev_t

In [ ]:
max_x = 250.
min_x = 5.
max_y = 100.
min_y = -100.
max_z = 1000.
min_z = 10.

fid_vol = (max_x-min_x)*(max_y-min_y)*(max_z-min_z)
print("Active volume: %f cm^3" % fid_vol)

ar_density=1.3836
ar_mmol = 39.95
av_k=6.02214e23
n_nucl=40

target_nucl=fid_vol*ar_density/ar_mmol*av_k*n_nucl
print("Target Nucleons: %.3e cm^3" % target_nucl)

In [ ]:
flux_per_pot = 7.379e-10
flux_tot = TOTAL_POT*flux_per_pot

print("Total Flux: %.3e cm^-2"%flux_tot)

In [ ]:
query_mucc = ("n_mu==1 and ccnc==0 and start_x_mu>%f and start_x_mu<%f and start_y_mu>%f and start_y_mu<%f and start_z_mu>%f and start_z_mu<%f"%(min_x,max_x,min_y,max_y,min_z,max_z))
df_mucc = df_ev_t.query(query_mucc).copy()
EVENTS_FID=len(df_mucc)
print("total events = %d"%EVENTS_FID)

In [ ]:
total_xsec = EVENTS_FID/(flux_tot*target_nucl)
print(total_xsec)

In [ ]:
df_mucc["costheta_mu"] = eval_costheta(df=df_mucc,suffix1="",suffix2="_mu")
df_mucc["q0"] = df_mucc["e"] - df_mucc["e_mu"]
df_mucc["q3"] = eval_q3(df=df_mucc,suffix1="",suffix2="_mu")

In [ ]:
df_mucc

In [ ]:
genie_all_err_theta = np.load("CCInclusive_XSec_Set2_NPY/genie_all_err_arr_theta.npy")
genie_all_err_p = np.load("CCInclusive_XSec_Set2_NPY/genie_all_err_arr.npy")
genie_oth_err_theta = np.load("CCInclusive_XSec_Set2_NPY/genie_other_arr_theta.npy")
genie_oth_err_p = np.load("CCInclusive_XSec_Set2_NPY/genie_other_arr.npy")

migr_matrix_p = np.load("CCInclusive_XSec_Set2_NPY/migration_matrix_momentum.npy")
migr_matrix_theta = np.load("CCInclusive_XSec_Set2_NPY/migration_matrix_theta.npy")

data_p = np.load("CCInclusive_XSec_Set2_NPY/data_points_momentum.npy")
data_theta = np.load("CCInclusive_XSec_Set2_NPY/data_points_theta.npy")

bkg_p = np.load("CCInclusive_XSec_Set2_NPY/mc_bkg_points_momentum.npy")
bkg_theta = np.load("CCInclusive_XSec_Set2_NPY/mc_bkg_points_theta.npy")

eff_reco_p = np.load("CCInclusive_XSec_Set2_NPY/eff_tilde_momentum.npy")
eff_reco_theta = np.load("CCInclusive_XSec_Set2_NPY/eff_tilde_theta.npy")
eff_true_theta = np.load("CCInclusive_XSec_Set2_NPY/true_efficiency_theta.npy")

stat_err_theta = np.sqrt(data_theta)
bins_theta = np.load("CCInclusive_XSec_Set2_NPY/theta_bins.npy")
total_err_theta = np.load("CCInclusive_XSec_Set2_NPY/genie_all_err_arr_theta.npy")

target_nucl_fd = 4.10331109202e+31
flux_fd = 3.54124623456e+20*7.3789785277e-10



In [ ]:
print(genie_all_err_theta)
print(stat_err_theta)
print(genie_oth_err_theta)
print(migr_matrix_theta)
print(data_theta)
print(bkg_theta)
print(data_theta-bkg_theta)

print(eff_reco_theta)
print(eff_true_theta)

print(bins_theta)


In [ ]:
#bins_theta = [ -1.00, -0.50, 0.00, 0.28, 0.47, 0.63, 0.765, 0.865, 0.935, 1.00 ]

fig,ax = plt.subplots()

plt.grid(axis='y', linewidth=0.5)
evs_cth,ar_cth,patches = plt.hist(df_mucc["costheta_mu"], color='red', label='$\mu$', bins=bins_theta,**pltops_hist)
print(evs_cth,ar_cth)
#plt.legend(loc=2)
plt.title("BNB $\\nu_\mu$ CC (GENIE Truth Study, Set2)")
plt.xlabel("Muon $cos(\\theta)$")
plt.ylabel("Events")
plt.show()
#plt.savefig("plots/genie_1mu1p_costheta.pdf")

In [ ]:
evs_cth_bn = np.array([ evs_cth[i]/(ar_cth[i+1]-ar_cth[i]) for i in range(len(evs_cth)) ])
evs_cth_bn_plt = np.append(evs_cth_bn,evs_cth_bn[-1])

evs_cth_sm = np.dot(migr_matrix_theta,evs_cth)
evs_cth_sm_bn = np.array([ evs_cth_sm[i]/(ar_cth[i+1]-ar_cth[i]) for i in range(len(evs_cth_sm)) ])
evs_cth_sm_bn_plt = np.append(evs_cth_sm_bn,evs_cth_sm_bn[-1])

print(np.sum(evs_cth))
print(np.sum(evs_cth_sm))

fig,ax = plt.subplots()
plt.plot(ar_cth,evs_cth_bn_plt,color='blue',linewidth=2.0,ds="steps-post",label="truth")
plt.plot(ar_cth,evs_cth_sm_bn_plt,color='black',linewidth=2.0,ds="steps-post",label="smeared")
plt.grid()
plt.show()

In [ ]:
print(evs_cth,"\n")
print(evs_cth*eff_true_theta,"\n")
print(np.dot(migr_matrix_theta,evs_cth*eff_true_theta),"\n")

print(np.dot(migr_matrix_theta,evs_cth),"\n")
print(np.dot(migr_matrix_theta,evs_cth)*eff_reco_theta,"\n")

In [ ]:
evs_cth_sm_eff = evs_cth_sm*eff_reco_theta

evs_cth_sm_plt = np.append(evs_cth_sm,evs_cth_sm[-1])
evs_cth_plt = np.append(evs_cth,evs_cth[-1])
evs_cth_sm_eff_plt = np.append(evs_cth_sm_eff,evs_cth_sm_eff[-1])

print(evs_cth_plt)
print(evs_cth_sm_plt)
print(evs_cth_sm_eff)

fig,ax = plt.subplots()
plt.plot(ar_cth,evs_cth_plt,color='blue',linewidth=2.0,ds="steps-post",label="truth")
plt.plot(ar_cth,evs_cth_sm_plt,color='black',linewidth=2.0,ds="steps-post",label="smeared")
plt.plot(ar_cth,evs_cth_sm_eff_plt,color='red',linewidth=2.0,ds="steps-post",label="efficiency applied")
plt.grid()
plt.show()

In [ ]:
#print(evs_cth_sm)
#print(ar_cth)
dxsec_dcth_tr = np.array([ evs_cth[i]/(ar_cth[i+1]-ar_cth[i])/(flux_tot*target_nucl) for i in range(len(evs_cth))])
dxsec_dcth = np.array([ evs_cth_sm[i]/(ar_cth[i+1]-ar_cth[i])/(flux_tot*target_nucl) for i in range(len(evs_cth))])
dxsec_dcth_fd = np.array([ (data_theta[i]-bkg_theta[i])/(ar_cth[i+1]-ar_cth[i])/(flux_fd*target_nucl_fd*eff_reco_theta[i]) for i in range(len(data_theta))])
dxsec_dcth_staterr_fd = np.array([ (stat_err_theta[i])/(ar_cth[i+1]-ar_cth[i])/(flux_fd*target_nucl_fd*eff_reco_theta[i]) for i in range(len(stat_err_theta))])
print(dxsec_dcth_tr)
print(dxsec_dcth)
print(dxsec_dcth_fd)
xsec_tot_dcth_check=0
xsec_tot_dcth_tr_check=0
xsec_tot_dcth_fd_check=0
for i in range(len(dxsec_dcth)):
    xsec_tot_dcth_check += dxsec_dcth[i]*(ar_cth[i+1]-ar_cth[i])
    xsec_tot_dcth_tr_check += dxsec_dcth_tr[i]*(ar_cth[i+1]-ar_cth[i])
    xsec_tot_dcth_fd_check += dxsec_dcth_fd[i]*(ar_cth[i+1]-ar_cth[i])    
print(xsec_tot_dcth_check,xsec_tot_dcth_tr_check,xsec_tot_dcth_fd_check)
print(xsec_tot_dcth_fd_check/xsec_tot_dcth_check)
print(dxsec_dcth_fd*genie_all_err_theta)

dxsec_dcth_fd_err = np.array([ np.sqrt(1e-38*genie_all_err_theta[i]*1e-38*genie_all_err_theta[i]+dxsec_dcth_staterr_fd[i]*dxsec_dcth_staterr_fd[i]) for i in range(len(stat_err_theta))])
dxsec_dcth_fd_xsec_err = np.array([ np.sqrt(1e-38*genie_all_err_theta[i]*1e-38*genie_all_err_theta[i]) for i in range(len(stat_err_theta))])
ar_cth_mid = np.array([ (ar_cth[i+1]+ar_cth[i])*0.5 for i in range(len(ar_cth)-1) ])
ar_cth_err = np.array([ (ar_cth[i+1]-ar_cth[i])*0.5 for i in range(len(ar_cth)-1) ])
print(ar_cth_mid)
print(ar_cth_err)
print(dxsec_dcth_fd_err/dxsec_dcth_fd)
print(dxsec_dcth_fd_xsec_err/dxsec_dcth_fd)
print(dxsec_dcth_staterr_fd/dxsec_dcth_fd)


In [ ]:
dxsec_dcth_plt = np.append(dxsec_dcth,dxsec_dcth[-1])
#dxsec_dcth_fd_plt=dxsec_dcth_fd.copy()
#dxsec_dcth_fd_plt.append(dxsec_dcth_fd[-1])

#ar_cth_mid = [ (ar_cth[i+1]+ar_cth[i])*0.5 for i in range(len(ar_cth)-1) ]

fig,ax = plt.subplots()
plt.plot(ar_cth,dxsec_dcth_plt,color='blue',linewidth=2.0,ds="steps-post",label="truth")
#plt.plot(ar_cth,dxsec_dcth_fd_plt,color='black',linewidth=2.0,ds="steps-post",label="fake data")
plt.errorbar(x=ar_cth_mid,y=dxsec_dcth_fd,xerr=ar_cth_err,yerr=dxsec_dcth_fd_err,fmt=".",color='black',label="fake data")
plt.grid()
#plt.legend()
#plt.title("LEE $\\nu_e$ signal model weights",fontsize=24)
#plt.xlabel("$E_\\nu$ (GeV)",fontsize=18)
#plt.ylabel("weight",fontsize=18)
plt.show()

In [ ]:
dxsec_dcth_diff = [ (dxsec_dcth_fd[i]/dxsec_dcth[i] - 1.0) for i in range(len(dxsec_dcth))]
dxsec_dcth_diff_err = [ (dxsec_dcth_fd_err[i]/dxsec_dcth[i]) for i in range(len(dxsec_dcth))]

fig,ax = plt.subplots()
plt.errorbar(x=ar_cth_mid,y=dxsec_dcth_diff,xerr=ar_cth_err,yerr=dxsec_dcth_diff_err,fmt=".",color='black',label="fake data")
plt.grid()
plt.show()

In [ ]:
bins_p = [ 0.00, 0.18, 0.30, 0.45, 0.77, 1.28, 2.50 ]

fig,ax = plt.subplots()

plt.grid(axis='y', linewidth=0.5)
evs_p,ar_p,patches = plt.hist(df_mucc["p_mu"], color='red', label='$\mu$', bins=bins_p,**pltops_hist)
#plt.legend()
plt.title("BNB $\\nu_\mu$ CC (GENIE Truth Study, Set2)")
plt.xlabel("Muon $p$")
plt.ylabel("Events")
plt.show()
#plt.savefig("plots/genie_1mu1p_costheta.pdf")

In [ ]:
print(evs_p)
print(ar_p)
dxsec_dp = [ evs_p[i]/(ar_p[i+1]-ar_p[i])/(flux_tot*target_nucl) for i in range(len(evs_p))]
print(dxsec_dp)
xsec_tot_dp_check=0
for i in range(len(dxsec_dp)):
    print(dxsec_dp[i]*(ar_p[i+1]-ar_p[i]))
    xsec_tot_dp_check += dxsec_dp[i]*(ar_p[i+1]-ar_p[i])
print(xsec_tot_dp_check)

In [ ]:
dxsec_dp_plt=dxsec_dp
dxsec_dp_plt.append(dxsec_dp[-1])

fig,ax = plt.subplots()
plt.plot(ar_p,dxsec_dp_plt,color='blue',linewidth=2.0,ds="steps-post")
plt.grid()
#plt.legend()
#plt.title("LEE $\\nu_e$ signal model weights",fontsize=24)
#plt.xlabel("$E_\\nu$ (GeV)",fontsize=18)
#plt.ylabel("weight",fontsize=18)
plt.show()

In [ ]:
bins_x = np.arange(-100,300,1)
bins_y = np.arange(-200,200,1)
bins_z = np.arange(-200,1500,1)

fig,axes = plt.subplots(3,1)

axes[0].grid(axis='y', linewidth=0.5)
axes[0].hist(df_mucc["start_x_mu"], color='red', label='$\mu$', bins=bins_x,**pltops_hist)
axes[1].grid(axis='y', linewidth=0.5)
axes[1].hist(df_mucc["start_y_mu"], color='red', label='$\mu$', bins=bins_y,**pltops_hist)
axes[2].grid(axis='y', linewidth=0.5)
axes[2].hist(df_mucc["start_z_mu"], color='red', label='$\mu$', bins=bins_z,**pltops_hist)
#plt.legend()
#plt.title("BNB $\\nu_\mu$ CC (GENIE Truth Study, Set2)")
#plt.xlabel("Muon $p$")
#plt.ylabel("Events")
plt.show()
#plt.savefig("plots/genie_1mu1p_costheta.pdf")